# Accumulating rain from several files

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pyart
import pylab as pl
import numpy as np
import glob
import os
import re
import wradlib as wrl
from datetime import datetime
#from wradlib.trafo import idecibel


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
filepath="MVOL/"

#filelist=(glob.glob(filepath+"SAO*"))

print filelist

In [32]:
#radar=[]
fileradar=[]
rain_timeseries=[]
for filename in glob.iglob(filepath+"SAO*"):
    #try:
        r=pyart.io.auto_read.read(filename)
        m=re.search('ppi', r.scan_type, flags=0)
        if m is not None:
            if ('reflectivity' and 'specific_differential_phase' 
                and 'cross_correlation_ratio') in r.fields.keys():
                #radar.append(r)
                gatefilter=pyart.correct.moment_based_gate_filter(r,min_refl=0.,min_rhv=0.8)
                gatefilter.exclude_outside(field='differential_reflectivity',v1=-2.,v2=6.)
                gatefilter.exclude_outside(field='velocity',v1=-50.,v2=50.)
                rain=pyart.retrieve.est_rain_rate_zkdp(r,alphaz=0.15,betaz=0.39,alphakdp=19.63,betakdp=0.823,master_field='reflectivity')
                rain_timeseries.append(rain['data'])
                fileradar.append(datetime.strptime(os.path.splitext(os.path.basename(filename))[0][3:],'%y%m%d%H%M%S'))
    #except:
    #    print "unreadable file",filename
    #    continue



## Daqui para frente, em construção !!

In [ ]:
dt_trg=[datetime.strptime('2016-06-05-00-00-00','%Y-%m-%d-%H-%M-%S')]
print dt_trg
aggr=wrl.util.aggregate_in_time(rain_timeseries, fileradar, dt_trg, taxis=0, func='sum')

In [33]:
print [len(rain_timeseries[i].data) for i in range(60)]

[359, 360, 359, 360, 360, 359, 360, 360, 359, 360, 360, 360, 360, 360, 359, 360, 360, 360, 360, 360, 360, 360, 360, 359, 360, 360, 360, 360, 360, 360, 360, 360, 360, 360, 360, 360, 356, 360, 360, 360, 360, 360, 360, 360, 360, 360, 360, 360, 360, 359, 359, 360, 360, 360, 360, 359, 360, 360, 360, 359]


801
